### Checks which sequences have been downloaded in folder `wd`

In [1]:
import sys
sys.path.append('/home/grg/git/xnat-monitor/')
import bbrc
from bbrc import xnat
import string, os
import os.path as osp
import numpy as np
from glob import glob

In [2]:
central = xnat.connect_xnat('/home/grg/.xnat_bsc.cfg')
#e = list(central.select.project('ALFA_PLUS').subject('66237').experiments())[0]
#xnat.check_session(central, e.label(), send_mail=False)

In [5]:
wd = '/tmp/downloads/NIFTI'
sequences = ['1501', '1601', '1701', '1301', '1401', '1201', '1101', '301', '401', '501', '701', '901']
subjects = [string.atoi(e) for e in open('/tmp/amyloid_subjects.txt').read().split('\n') if e != '']

In [5]:
t = []
for subject in subjects:
    t.append([subject])
    n = 0
    for seq in sequences:
        fp = osp.join(wd, '%s_%s.nii.gz'%(subject, seq))
        t[-1].append(osp.isfile(fp))
        n = n + osp.isfile(fp)
    t[-1].append(n)
        
import pandas as pd
df = pd.DataFrame(t, columns=['subject', '1501', '1601', '1701', '1301', '1401', '1201', '1101', '301', '401', '501', '701', '901', 'n'])
duplicates = df[df['n']!=2]['subject'].tolist()
for each in duplicates:
    print each
    expids = xnat.subject_experiments(central, 'ALFA_PLUS', str(each))
    print expids
    d = xnat.series_desc(central, expids[0].id())
    seq = list(np.array(sequences)[df[df['subject']==each].iloc[0].tolist()[1:-1]])
    print [(s, d[s]) for s in seq]
        

55854
[<Experiment Object> BBRC02_E00294]
[]


### Subjects with wrong numbers of images

In [33]:
df[df['n']!=2]

,subject,1501,1601,1701,1301,1401,1201,1101,301,401,501,701,901,n
7,10225,False,False,False,True,True,False,False,False,True,False,False,False,3
35,11273,False,False,False,False,True,False,False,False,True,False,False,True,3
40,11514,True,True,False,False,False,False,False,False,True,False,False,False,3
41,11540,False,False,False,True,False,False,False,True,True,False,False,False,3
63,12991,True,False,False,False,True,False,False,False,True,False,False,False,3
69,13166,False,False,False,True,True,False,False,False,True,False,False,False,3
84,44363,False,False,False,True,True,False,False,True,False,False,False,False,3
87,55038,False,False,False,True,True,False,False,False,True,False,False,False,3
94,55854,False,False,False,False,True,True,False,True,False,False,False,False,3
100,66162,True,False,False,False,True,False,False,False,True,False,False,False,3


### Creates a clean tree with files

In [33]:
wd = '/tmp/amyloid2/NIFTI'
wd2 = '/tmp/AmylStaging/'

import os
for s in subjects:
    print s
    files = [e.split('_')[1].split('.')[0] for e in glob(osp.join(wd, '%s_*'%s)) if not e.endswith('.json')]
    count = [files.count(e) for e in set(files)]
    d = dict(zip(count, set(files)))
    dtiseq, t1seq = d[3], d[1]
    t1fp = osp.join(wd, '%s_%s.nii.gz'%(s, t1seq))
    dwi, bval, bvec = [osp.join(wd, '%s_%s.%s'%(s, dtiseq, e)) for e in ['nii.gz', 'bval', 'bvec']]
    print s, d, t1fp, dwi, bval, bvec
    subd = osp.join(wd2, str(s))
    commands = []
    commands.append('mkdir %s'%subd)
    commands.append('mkdir %s'%osp.join(subd, 'T1'))
    commands.append('mkdir %s'%osp.join(subd, 'DWI'))
    
    cmd = 'cp %s %s'%(t1fp, osp.join(wd2, str(s), 'T1', '%s.nii.gz'%s))
    commands.append(cmd)
    cmd = 'cp %s %s'%(dwi, osp.join(wd2, str(s), 'DWI', '%s.nii.gz'%s))
    commands.append(cmd)
    cmd = 'cp %s %s'%(bval, osp.join(wd2, str(s), 'DWI', '%s.bval'%s))
    commands.append(cmd)
    cmd = 'cp %s %s'%(bvec, osp.join(wd2, str(s), 'DWI', '%s.bvec'%s))
    commands.append(cmd)
    
    #print commands
    for cmd in commands:
        print cmd
        os.system(cmd)
    
    
    

10019
10019 {1: '301', 3: '1301'} /tmp/amyloid2/NIFTI/10019_301.nii.gz /tmp/amyloid2/NIFTI/10019_1301.nii.gz /tmp/amyloid2/NIFTI/10019_1301.bval /tmp/amyloid2/NIFTI/10019_1301.bvec
mkdir /tmp/AmylStaging/10019
mkdir /tmp/AmylStaging/10019/T1
mkdir /tmp/AmylStaging/10019/DWI
cp /tmp/amyloid2/NIFTI/10019_301.nii.gz /tmp/AmylStaging/10019/T1/10019.nii.gz
cp /tmp/amyloid2/NIFTI/10019_1301.nii.gz /tmp/AmylStaging/10019/DWI/10019.nii.gz
cp /tmp/amyloid2/NIFTI/10019_1301.bval /tmp/AmylStaging/10019/DWI/10019.bval
cp /tmp/amyloid2/NIFTI/10019_1301.bvec /tmp/AmylStaging/10019/DWI/10019.bvec
10044
10044 {1: '401', 3: '1301'} /tmp/amyloid2/NIFTI/10044_401.nii.gz /tmp/amyloid2/NIFTI/10044_1301.nii.gz /tmp/amyloid2/NIFTI/10044_1301.bval /tmp/amyloid2/NIFTI/10044_1301.bvec
mkdir /tmp/AmylStaging/10044
mkdir /tmp/AmylStaging/10044/T1
mkdir /tmp/AmylStaging/10044/DWI
cp /tmp/amyloid2/NIFTI/10044_401.nii.gz /tmp/AmylStaging/10044/T1/10044.nii.gz
cp /tmp/amyloid2/NIFTI/10044_1301.nii.gz /tmp/AmylStaging

### Building commands for denoising

In [11]:
import sys 
sys.path.append('/home/grg/git/thesaurus/')
import thesaurus as th
jf = '/home/grg/git/thesaurus/alfa_dwi_pipeline_aug2016.json'
dd = '/home/grg/data/AmylStaging'
json_io = '/home/grg/git/thesaurus/alfa_dwi_pipeline_io_aug2016.json'
a = th.ALFAHelper(directory=dd, jsonfile=json_io)

cmds = []
for e in ['denoising']:
    for s in subjects:  
        cmd = a.parse_command(str(s), e, jsonfile=jf)
        cmds.append(cmd)
 
# Writing commands in a textfile
fp1 = '/tmp/denoising.sh'
fp2 = '/tmp/denoising.workflow'
w = open(fp1, 'w')
for e in cmds:
    w.write('%s\n'%e)
w.close()

import sys
sys.path.append('/home/grg/git/alfa')
import create_workflow as cw
names = [str(e) for e in subjects]
#names.extend([str(e) for e in subjects])

cw.create_workflow(fp1, fp2, names=names, verbose=True)

[['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/AmylStaging/13193/DWI/13193.nii'], ['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/AmylStaging/13244/DWI/13244.nii'], ['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/AmylStaging/13259/DWI/13259.nii'], ['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/AmylStaging/21136/DWI/21136.nii'], ['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/AmylStaging/21138/DWI/21138.nii'], ['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/AmylStaging/21156/DWI/21156.nii'], ['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/AmylStaging/44008/DWI/44008.nii'], ['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/AmylStaging/44158/DWI/44158.nii'], ['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/AmylStaging/44214/DWI/44214.nii.gz'], ['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/A

### Retrieving failed ones (DWI)

In [4]:
import sys 
sys.path.append('/home/grg/git/thesaurus/')
import thesaurus as th
jf = '/home/grg/git/thesaurus/alfa_dwi_pipeline_aug2016.json'
dd = '/home/grg/data/AmylStaging'
json_io = '/home/grg/git/thesaurus/alfa_dwi_pipeline_io_aug2016.json'
a = th.ALFAHelper(directory=dd, jsonfile=json_io)
subjects = [12118, 13041, 55854, 10162, 10515, 10627, 10649, 10711, 10949, 11262, 11514, 11745, 12279, 12741, 12991, 13035, 13105, 13107, 13193, 13259, 
             21136, 21138, 44008, 44301, 44396, 66008, 66027, 66100, 66162, 66301, 77012, 77164]

cmds = []
for e in ['denoising']:
    for s in subjects:  
        cmd = a.parse_command(str(s), e, jsonfile=jf)
        cmds.append(cmd)
 
# Writing commands in a textfile
fp1 = '/tmp/failed_denoising.sh'
fp2 = '/tmp/failed_denoising.workflow'
w = open(fp1, 'w')
for e in cmds:
    w.write('%s\n'%e)
w.close()

import sys
sys.path.append('/home/grg/git/alfa')
import create_workflow as cw
names = [str(e) for e in subjects]

cw.create_workflow(fp1, fp2, names=names, verbose=True)
print('Done for %s subjects'%len(subjects))

[['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/AmylStaging/12118/DWI/12118.nii'], ['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/AmylStaging/13041/DWI/13041.nii'], ['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/AmylStaging/55854/DWI/55854.nii'], ['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/AmylStaging/10162/DWI/10162.nii'], ['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/AmylStaging/10515/DWI/10515.nii'], ['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/AmylStaging/10627/DWI/10627.nii'], ['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/AmylStaging/10649/DWI/10649.nii'], ['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/AmylStaging/10711/DWI/10711.nii'], ['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/AmylStaging/10949/DWI/10949.nii'], ['python', '/home/grg/git/alfa/denoising.py', '-i', '/home/grg/data/Amyl